In [2]:
import os
import math
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt


In [3]:
# single household contsumption data extract function
def extract_house(data_frame, lclid):
    df = data_frame.set_index("tstp")
    df.index = df.index.astype("datetime64")
    
    df["energy(kWh/hh)"] = df["energy(kWh/hh)"].astype("float64")
    df = df[df["LCLid"] == "{}".format(lclid) ]
    return df

# weather extraction function
def extract_weather(data_frame):
    df = data_frame.set_index("time")
    df.index = df.index.astype("datetime64")
    return df

### Electricity consumption processing

In [4]:
elec = pd.read_csv("smart-meters-in-london/halfhourly_dataset/block_0.csv")
for num in range(1,111):    
    elec = elec.append(pd.read_csv("smart-meters-in-london/halfhourly_dataset/block_"+str(num)+".csv"))
        

//miniconda3/envs/TPOT/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
elec.head(1)

,LCLid,tstp,energy(kWh/hh),energy
0,MAC000002,2012-10-12 00:30:00.0000000,0,0.0


In [18]:
elec.replace('Null', np.NaN, inplace=True)
energy = elec
energy["energy"] = energy["energy(kWh/hh)"].astype("float64")
energy = energy.drop(["energy(kWh/hh)"],axis = 1)

**Missing values**

In [18]:
def missing_count(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: data[df_cols[i]].isnull().sum()})
    
    return dict_x


missing = missing_count(energy)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Count of missing data')
df_miss[0:10]


Count of missing data


[('energy', 5544), ('LCLid', 0), ('tstp', 0)]

**Group by LCLid and aggregate over per hour**


In [19]:
## deal with missing value by interpolating mean of whole column
energy["energy"].fillna(energy["energy"].median(), inplace = True)
## group by time stamps and aggregate through mean
energy["energy"] = energy["energy"].astype("float64")
energy["tstp"] = energy["tstp"].astype("datetime64")
energy = energy.groupby('tstp').mean()
energy['tstp'] = energy.index
energy.drop(['tstp'], axis = 1, inplace=True)
energy = energy.resample('1H').sum()

### Weather data processing


In [105]:
weather = pd.read_csv('smart-meters-in-london/weather_hourly_darksky.csv')
weather.head(10)

,visibility,windBearing,temperature,time,dewPoint,pressure,apparentTemperature,windSpeed,precipType,icon,humidity,summary
0,5.97,104,10.24,2011-11-11 00:00:00,8.86,1016.76,10.24,2.77,rain,partly-cloudy-night,0.91,Partly Cloudy
1,4.88,99,9.76,2011-11-11 01:00:00,8.83,1016.63,8.24,2.95,rain,partly-cloudy-night,0.94,Partly Cloudy
2,3.70,98,9.46,2011-11-11 02:00:00,8.79,1016.36,7.76,3.17,rain,partly-cloudy-night,0.96,Partly Cloudy
3,3.12,99,9.23,2011-11-11 03:00:00,8.63,1016.28,7.44,3.25,rain,fog,0.96,Foggy
4,1.85,111,9.26,2011-11-11 04:00:00,9.21,1015.98,7.24,3.70,rain,fog,1.00,Foggy
5,1.96,115,9.33,2011-11-11 05:00:00,8.87,1015.91,7.19,3.97,rain,fog,0.97,Foggy
6,1.30,118,9.31,2011-11-11 06:00:00,8.82,1015.70,7.10,4.10,rain,fog,0.97,Foggy
7,1.22,114,8.85,2011-11-11 07:00:00,8.69,1016.08,6.48,4.23,rain,fog,0.99,Foggy
8,1.40,120,9.13,2011-11-11 08:00:00,8.75,1016.33,6.84,4.20,rain,fog,0.97,Foggy
9,1.38,121,9.23,2011-11-11 09:00:00,8.70,1016.57,7.07,3.96,rain,fog,0.97,Foggy


In [70]:
weather.dtypes

visibility             float64
windBearing              int64
temperature            float64
time                    object
dewPoint               float64
pressure               float64
apparentTemperature    float64
windSpeed              float64
precipType              object
icon                    object
humidity               float64
summary                 object
dtype: object

**Missing values**

In [92]:
def count_missing(df):
    data = pd.DataFrame(df)
    df_cols = list(pd.DataFrame(data))
    dict_x = {}
    for i in range(0, len(df_cols)):
        dict_x.update({df_cols[i]: round(data[df_cols[i]].isnull().sum()*100,2)})
    
    return dict_x

missing = count_missing(weather)
df_miss = sorted(missing.items(), key=lambda x: x[1], reverse=True)
print('Percent of missing data')
df_miss[0:]

Percent of missing data


[('summary', 0),
 ('pressure', 0),
 ('dewPoint', 0),
 ('icon', 0),
 ('visibility', 0),
 ('temperature', 0),
 ('time', 0),
 ('precipType', 0),
 ('windSpeed', 0),
 ('apparentTemperature', 0),
 ('windBearing', 0),
 ('humidity', 0)]

In [106]:
## fillna with median
weather["pressure"] = weather["pressure"].fillna(weather["pressure"].median())

**One hot encoding**

In [73]:
## Examine unique categories in categorical features
p = list(weather["precipType"].unique()) 
i = list(weather["icon"].unique()) 
u = list(weather["summary"].unique())
# Sort them for one hot encoding labels
p.sort()
i.sort()
u.sort()

In [74]:
from sklearn.preprocessing import OneHotEncoder
## One hot encoding these features above
onehot = OneHotEncoder(dtype=np.int, sparse=True)
nominals = pd.DataFrame(onehot.fit_transform(weather[['precipType', 'icon','summary']]).toarray(),columns=p+i+u)
nominals.index = weather.index

In [129]:
#Concate nominals with origin weather dataframe
weather = weather.set_index("time")
weather.index = weather.index.astype("datetime64")
weather.drop(["precipType","icon","summary"],axis = 1, inplace = True)
weather =  weather.merge(nominals, left_index = True, right_index = True)


### Merge weather and energy data

In [175]:
### Merge weather and energy data together
weather_energy = weather.merge(energy, left_index = True, right_index = True)
weather_energy.index.names = ["time"]

In [177]:
##Split data into training set and testing set and save to csv file format
train_we = weather_energy.iloc[:14000,:]
test_we = weather_energy.iloc[14000:,:]
train_we.to_csv("train.csv", index = True)
test_we.to_csv("test.csv", index = True)

In [176]:
weather_energy

,visibility,windBearing,temperature,dewPoint,pressure,apparentTemperature,windSpeed,humidity,rain,snow,...,Clear,Foggy,Mostly Cloudy,Overcast,Partly Cloudy,Windy,Windy and Mostly Cloudy,Windy and Overcast,Windy and Partly Cloudy,energy
time,,,,,,,,,,,,,,,,,,,,,
2011-12-11 00:00:00,12.50,210,2.83,1.17,1015.67,1.11,1.78,0.89,1,0,...,0,0,0,0,1,0,0,0,0,0.498395
2011-12-11 01:00:00,12.65,204,2.48,0.81,1014.96,0.31,2.11,0.89,1,0,...,1,0,0,0,0,0,0,0,0,0.445535
2011-12-11 02:00:00,13.02,214,2.70,1.29,1014.42,0.11,2.57,0.90,1,0,...,1,0,0,0,0,0,0,0,0,0.349090
2011-12-11 03:00:00,13.05,211,3.47,1.41,1013.78,0.66,3.00,0.86,1,0,...,0,0,0,0,1,0,0,0,0,0.317786
2011-12-11 04:00:00,12.97,204,3.74,1.53,1012.94,1.29,2.64,0.85,1,0,...,0,0,0,0,1,0,0,0,0,0.281382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-02-15 19:00:00,12.68,247,7.39,3.06,997.07,3.91,6.08,0.74,1,0,...,0,0,0,0,1,0,0,0,0,0.744099
2014-02-15 20:00:00,13.78,249,6.56,2.76,998.15,3.03,5.61,0.77,1,0,...,1,0,0,0,0,0,0,0,0,0.685877
2014-02-15 21:00:00,14.31,248,6.47,2.71,999.28,3.06,5.25,0.77,1,0,...,1,0,0,0,0,0,0,0,0,0.634185
